In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import sys
sys.path.insert(0,'../python/')

import correlation_new.correlation_computer as cc

## Util functions

In [ ]:
def load_score_map(day, measure):
    """The centrality maps were pre-sorted in decreasing order!!!"""
    scores = pd.read_csv(data_path + '/%s_scores_%i.txt_s' % (measure,day), sep=" ", names=["id","score"])
    scores = scores.set_index("id")
    return scores

In [ ]:
def result2file(result_list,file_name):
    with open(file_name, 'w') as f:
        for i in xrange(len(result_list)):
            f.write('%i %f\n' % (i, result_list[i]))
    print 'Done'

In [ ]:
def calculate_corr_for_a_day(day, corr_type, measure):
    prev_day = load_score_map(day-1, measure)
    current_day = load_score_map(day, measure)
    if corr_type=="pearson":
        return cc.corr_pearson(prev_day,current_day)[0]
    elif corr_type=="spearman":
        return cc.corr_spearman(prev_day,current_day)[0]
    elif corr_type=="kendall":
        return cc.corr_kendalltau(prev_day,current_day)[0]
    elif corr_type=="w_kendall":
        return cc.corr_weighted_kendalltau(prev_day,current_day)[0]
    else:
        raise RuntimeError("Invalid correlation type: %s!" % corr_type)
        
def calculate_corr_for_days(days, corr_type, measure_type):
    return map(lambda x: calculate_corr_for_a_day(x,corr_type=corr_type, measure=measure_type), days)

## Compute correlation

In [ ]:
dataset_id = '15o'
measure_id = 'pagerank'
data_path = '/mnt/idms/fberes/NETWORK/DATA/temporal_centralities/centrality_output_for_datasets/%s/centrality_scores/' % dataset_id
output_path = '/mnt/idms/fberes/NETWORK/andreas_article/correlations/'

days = range(1,22)
print days

In [ ]:
%%time
pearson_result = calculate_corr_for_days(days,corr_type="pearson",measure_type=measure_id)
result2file(pearson_result, output_path + "/%s_%s.pearson" % (dataset_id,measure_id))
print pearson_result

In [ ]:
%%time
spearman_result = calculate_corr_for_days(days,corr_type="spearman",measure_type=measure_id)
result2file(spearman_result, output_path + "/%s_%s.spearman" % (dataset_id,measure_id))
print spearman_result

In [ ]:
%%time
kendall_result = calculate_corr_for_days(days,corr_type="kendall",measure_type=measure_id)
result2file(kendall_result, output_path + "/%s_%s.kendall" % (dataset_id,measure_id))
print kendall_result

%%time
calculate_corr_for_days(days,corr_type="w_kendall")

## Vizualization